In [7]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('ml-latest-small/movies.csv')
ratings_df = pd.read_csv('ml-latest-small/ratings.csv')

In [8]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [9]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [11]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [12]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

In [16]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [17]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0468, 0.0482, 0.0231,  ..., 0.0416, 0.0040, 0.0139],
        [0.0191, 0.0233, 0.0024,  ..., 0.0424, 0.0163, 0.0483],
        [0.0267, 0.0081, 0.0410,  ..., 0.0044, 0.0364, 0.0140],
        ...,
        [0.0382, 0.0084, 0.0202,  ..., 0.0213, 0.0395, 0.0221],
        [0.0498, 0.0447, 0.0269,  ..., 0.0204, 0.0038, 0.0025],
        [0.0383, 0.0205, 0.0331,  ..., 0.0476, 0.0461, 0.0229]])
item_factors.weight tensor([[0.0339, 0.0128, 0.0094,  ..., 0.0315, 0.0440, 0.0129],
        [0.0494, 0.0316, 0.0466,  ..., 0.0397, 0.0251, 0.0333],
        [0.0430, 0.0408, 0.0231,  ..., 0.0101, 0.0344, 0.0359],
        ...,
        [0.0463, 0.0120, 0.0151,  ..., 0.0195, 0.0227, 0.0251],
        [0.0400, 0.0413, 0.0305,  ..., 0.0142, 0.0246, 0.0499],
        [0.0436, 0.0413, 0.0498,  ..., 0.0411, 0.0240, 0.0405]])


In [22]:
from tqdm import tqdm

for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))


iter #0 Loss: 11.052578601135215


iter #1 Loss: 4.737398703691318


iter #2 Loss: 2.4745075167133117


iter #3 Loss: 1.7213465911785357


iter #4 Loss: 1.3455439713703194


...

iter #124 Loss: 0.32726537687751245


iter #125 Loss: 0.32705523024506983


iter #126 Loss: 0.3263539454431703




100%|██████████| 128/128 [08:25<00:00,  3.95s/it]

iter #127 Loss: 0.326052902786411


In [23]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.7681,  1.9529,  0.3418,  ...,  2.0451,  1.1168,  0.6876],
        [ 0.8816,  1.3169,  1.4312,  ...,  0.5131,  0.3093,  2.5003],
        [ 0.1158, -3.1533,  1.3337,  ...,  2.2951, -0.5305,  0.4173],
        ...,
        [ 1.4990, -0.3579,  0.4343,  ...,  0.8379,  0.8630,  2.0458],
        [ 0.8730,  1.3238,  0.2725,  ...,  1.4177,  0.8391,  1.3517],
        [ 1.6905,  0.3419,  1.2247,  ...,  1.5976,  0.6017,  0.8488]])
item_factors.weight tensor([[ 0.4038,  0.6276,  0.7385,  ...,  0.4941,  0.4632,  0.2899],
        [-0.0719,  0.5910, -0.0098,  ...,  0.5960,  0.8680,  0.3174],
        [ 0.7157,  0.2274,  0.3898,  ...,  0.5943,  0.4019,  0.5988],
        ...,
        [ 0.3570,  0.3405,  0.3245,  ...,  0.3291,  0.3328,  0.3336],
        [ 0.3995,  0.4154,  0.3892,  ...,  0.3728,  0.3874,  0.4089],
        [ 0.4126,  0.4248,  0.4168,  ...,  0.4088,  0.3890,  0.4065]])


In [24]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [25]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [26]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [31]:
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        rat_count = ratings_df.loc[ratings_df['movieId']==movid].iloc[:, 0].count()
        movs.append((movie_names[movid], rat_count))
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 GoldenEye (1995)
	 Net, The (1995)
	 Crimson Tide (1995)
	 Cliffhanger (1993)
	 Happy Gilmore (1996)
	 Meet the Parents (2000)
	 Broken Arrow (1996)
	 Nutty Professor, The (1996)
	 Demolition Man (1993)
	 Matrix Revolutions, The (2003)
Cluster #1
	 Star Wars: Episode IV - A New Hope (1977)
	 Fight Club (1999)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Star Wars: Episode VI - Return of the Jedi (1983)
	 Fargo (1996)
	 Ace Ventura: Pet Detective (1994)
	 Groundhog Day (1993)
	 Indiana Jones and the Last Crusade (1989)
	 Monty Python and the Holy Grail (1975)
	 One Flew Over the Cuckoo's Nest (1975)
Cluster #2
	 Batman & Robin (1997)
	 Showgirls (1995)
	 Godzilla (1998)
	 Barb Wire (1996)
	 Rocky V (1990)
	 Superman IV: The Quest for Peace (1987)
	 Karate Kid, Part III, The (1989)
	 Vertical Limit (2000)
	 Mimic (1997)
	 Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994)
Cluster #3
	 Jurassic Park (1993)
	 Braveheart (1995)
	 Terminato